# XAI Neural Net Creation

## setup

In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import os
import sys
currDir = os.path.dirname(os.path.realpath("__file__"))
rootDir = os.path.abspath(os.path.join(currDir, '..'))
sys.path.insert(1, rootDir)

In [40]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop,Adam
from tensorflow.keras.utils import to_categorical

from src.data.make_dataset import make_dataset_1, make_dataset_2
from src.utils.file_utils import save_df, save_model

## generate artificial dataset

In [32]:
df1 = make_dataset_1()
df2 = make_dataset_2()
save_df(df1, "dataset1.csv")
save_df(df2, "dataset2.csv")

successfully generated dataset | num_rows: 2600000
successfully generated dataset | num_rows: 504000
df successfully saved | filename: dataset1.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data
df successfully saved | filename: dataset2.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data


In [5]:
df2.describe()
df1.describe()

,mode,ei,to,td,tf,vers,consumption
count,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06,2.600000e+06
mean,1.160000e+01,1.510916e+00,2.940000e+01,6.505000e+01,1.050000e+01,4.500000e+00,1.484044e+02
std,8.002501e+00,1.132716e+00,3.891838e+01,3.752776e+01,5.766282e+00,2.872282e+00,3.282216e+02
min,5.000000e+00,5.899000e-01,1.000000e+00,1.000000e-01,1.000000e+00,0.000000e+00,6.479167e-06
25%,5.000000e+00,9.330000e-01,3.000000e+00,3.257500e+01,5.750000e+00,2.000000e+00,6.713139e+00
50%,6.500000e+00,1.354955e+00,4.500000e+00,6.505000e+01,1.050000e+01,4.500000e+00,3.509333e+01
75%,2.100000e+01,1.354955e+00,5.500000e+01,9.752500e+01,1.525000e+01,7.000000e+00,1.428353e+02
max,2.600000e+01,4.814286e+00,1.200000e+02,1.300000e+02,2.000000e+01,9.000000e+00,6.292000e+03


In [4]:
from src.utils.file_utils import load_df
df1, df2 = load_df("dataset1.csv"), load_df("dataset2.csv")

## create neural nets

In [33]:
# def build_model():    
#     model = Sequential()
#     model.add(Dense(6, input_dim=5, activation="relu"))#hidden layer
#     model.add(Dense(10, activation="relu"))#hidden layer
#     model.add(Dense(1, activation='sigmoid'))#output layer

#     optimizer = RMSprop(0.001)
#     model.compile(loss='mse', optimizer=optimizer, metrics=['mse', 'mae', 'mape'])
#     return model

def build_model(num_features, num_classes):
    model = Sequential()
    model.add(Dense(32, input_shape=(num_features,), activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', # Cross-entropy
                optimizer='adam', metrics=['accuracy'])
    return model

## train

In [36]:
def prepare_df(df, y_column):
    temp_df = df.copy()
    temp_df.sample(frac=1)
    
    y = to_categorical(
        temp_df[y_column].values)
    temp_df.drop(y_column, axis=1, inplace=True)
    features = list(temp_df.columns)
    
    x = temp_df.values
    scaler = StandardScaler()
    scaler.fit(x)
    x = scaler.transform(x)
    
    return x,y,features

x1, y1, features1 = prepare_df(df1, "vers")
x_train1, x_test1, y_train1, y_test1 = train_test_split(x1, y1, test_size=0.20)
norm_train1 = pd.DataFrame(x_train1, columns=features1)
save_df(norm_train1, "norm_train1.csv")

x2, y2, features2 = prepare_df(df2, "vers")
x_train2, x_test2, y_train2, y_test2 = train_test_split(x2, y2, test_size=0.20)
norm_train2 = pd.DataFrame(x_train2, columns=features2)
save_df(norm_train2, "norm_train2.csv")

df successfully saved | filename: norm_train1.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data
df successfully saved | filename: norm_train2.csv, dir: /Users/test/Documents/GitHub/xai-senior-design/data


In [37]:
model1 = build_model(6,10)
history1 = model1.fit(x_train1, y_train1, epochs=10, validation_split=0.2, batch_size=128, verbose=2)

Train on 1664000 samples, validate on 416000 samples
Epoch 1/10
1664000/1664000 - 23s - loss: 1.1007 - accuracy: 0.6112 - val_loss: 0.7781 - val_accuracy: 0.7277
Epoch 2/10
1664000/1664000 - 23s - loss: 0.6800 - accuracy: 0.7564 - val_loss: 0.6077 - val_accuracy: 0.7804
Epoch 3/10
1664000/1664000 - 20s - loss: 0.5602 - accuracy: 0.7969 - val_loss: 0.5205 - val_accuracy: 0.8078
Epoch 4/10
1664000/1664000 - 21s - loss: 0.4945 - accuracy: 0.8209 - val_loss: 0.4611 - val_accuracy: 0.8233
Epoch 5/10
1664000/1664000 - 21s - loss: 0.4516 - accuracy: 0.8368 - val_loss: 0.4466 - val_accuracy: 0.8295
Epoch 6/10
1664000/1664000 - 22s - loss: 0.4252 - accuracy: 0.8459 - val_loss: 0.4090 - val_accuracy: 0.8410
Epoch 7/10
1664000/1664000 - 25s - loss: 0.4058 - accuracy: 0.8525 - val_loss: 0.3960 - val_accuracy: 0.8591
Epoch 8/10
1664000/1664000 - 23s - loss: 0.3917 - accuracy: 0.8575 - val_loss: 0.3927 - val_accuracy: 0.8521
Epoch 9/10
1664000/1664000 - 23s - loss: 0.3795 - accuracy: 0.8616 - val_lo

In [38]:
model2 = build_model(5,7)
history = model2.fit(x_train2, y_train2, epochs=15, validation_split=0.2, batch_size=128, verbose=2)

Train on 322560 samples, validate on 80640 samples
Epoch 1/15
322560/322560 - 5s - loss: 1.3114 - accuracy: 0.4340 - val_loss: 1.0834 - val_accuracy: 0.4988
Epoch 2/15
322560/322560 - 4s - loss: 0.9869 - accuracy: 0.5406 - val_loss: 0.9219 - val_accuracy: 0.5326
Epoch 3/15
322560/322560 - 6s - loss: 0.8848 - accuracy: 0.5891 - val_loss: 0.8505 - val_accuracy: 0.5762
Epoch 4/15
322560/322560 - 5s - loss: 0.8267 - accuracy: 0.6320 - val_loss: 0.8021 - val_accuracy: 0.6730
Epoch 5/15
322560/322560 - 5s - loss: 0.7798 - accuracy: 0.6696 - val_loss: 0.7572 - val_accuracy: 0.7079
Epoch 6/15
322560/322560 - 5s - loss: 0.7375 - accuracy: 0.6939 - val_loss: 0.7128 - val_accuracy: 0.7045
Epoch 7/15
322560/322560 - 4s - loss: 0.6995 - accuracy: 0.7193 - val_loss: 0.6724 - val_accuracy: 0.7484
Epoch 8/15
322560/322560 - 4s - loss: 0.6654 - accuracy: 0.7428 - val_loss: 0.6387 - val_accuracy: 0.7780
Epoch 9/15
322560/322560 - 4s - loss: 0.6351 - accuracy: 0.7605 - val_loss: 0.6237 - val_accuracy: 0.

## test

In [39]:
print(model1.evaluate(x_test1, y_test1))
print(model2.evaluate(x_test2, y_test2))
# [0.09895923781607832, 0.85723215]

520000/520000 [==============================] - 15s 29us/sample - loss: 0.3637 - accuracy: 0.8626
[0.3636828694964831, 0.86261344]
100800/100800 [==============================] - 3s 30us/sample - loss: 0.4701 - accuracy: 0.8293
[0.47007398725975125, 0.82925594]


In [35]:
preds = model2.predict_classes(x_test2)
# print(preds)
for i in range(len(y_test2[:10])):
    print(preds[i], list(y_test2[i]).index(1))

8 8
4 4
7 2
4 4
9 8
2 2
4 4
6 6
5 9
0 2


In [41]:
save_model(model1, "model1.h5")
save_model(model2, "model2.h5")

model successfully saved | file_location: /Users/test/Documents/GitHub/xai-senior-design/models/model1.h5
model successfully saved | file_location: /Users/test/Documents/GitHub/xai-senior-design/models/model2.h5
